In [ ]:
#用贝叶斯优化随机森林参数
from bayes_opt import BayesianOptimization  
 
from sklearn.model_selection import train_test_split, KFold, cross_val_score  
from sklearn.ensemble import RandomForestRegressor  
import numpy as np


# 数据集分割
x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.2, random_state=42)
#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
#x_train = X_train1
#y_train = y_train1
# 定义目标函数，使用交叉验证来计算 R^2
def objective(n_estimators, max_depth, min_samples_split):  
    # 创建随机森林回归器
    reg = RandomForestRegressor(
        n_estimators=int(n_estimators), 
        max_depth=int(max_depth), 
        min_samples_split=int(min_samples_split), 
        random_state=42
    )
    
    # 使用 KFold 进行交叉验证
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    results = cross_val_score(reg, x_train, y_train, cv=kfold, scoring='r2')
    
    # 返回平均 R^2 值
    return np.mean(results)

# 定义参数空间
pbounds = {
    'n_estimators': (5, 300), 
    'max_depth': (1, 20), 
    'min_samples_split': (2, 20)

  
}

# 进行贝叶斯优化
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=3
)

optimizer.maximize(
    init_points=20,  # 随机搜索的次数
    n_iter=50       # 总共优化的次数
)

# 输出结果
print(f"Best parameters found: {optimizer.max['params']}")
print(f"Best score found: {optimizer.max['target']}")